# Project - Airline AI Assistant with Multiple Tools

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline using 2 different tools for now, can be improved to use more like:
1. Get prices from a 3rd part api to set prices using scraper
2. Improve database to have a table reservation and a join table to reserve tickets, then add a tool to reserve tickets

In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import sqlite3

In [2]:
# Initialization
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4.1-mini"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [4]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool called for city {destination_city}")
    price = ticket_prices.get(destination_city.lower(), "Unknown ticket price")
    return f"The price of a ticket to {destination_city} is {price}"


In [5]:
def set_ticket_prices(cities, prices):
    ticket_prices = {}
    for city, price in zip(cities, prices):
        ticket_prices[city.lower()] = f"${price}"
    return ticket_prices

# Example usage:
cities = ["London", "Paris", "Tokyo", "Berlin"]
prices = [899, 999, 1500, 599]

ticket_prices = set_ticket_prices(cities, prices)
print(ticket_prices)


{'london': '$899', 'paris': '$999', 'tokyo': '$1500', 'berlin': '$599'}


In [6]:
get_ticket_price("London")

Tool called for city London


'The price of a ticket to London is $899'

In [7]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

set_price_function = {
  "name": "set_ticket_price",
  "description": "Set the prices of return tickets for multiple destination cities.",
  "parameters": {
    "type": "object",
    "properties": {
      "ticket_prices": {
        "type": "object",
        "description": "A dictionary where the key is the destination city and the value is the ticket price.",
        "additionalProperties": {
          "type": "string",
          "description": "The ticket price (formatted as a string, e.g., '$799')."
        }
      }
    },
    "required": ["ticket_prices"],
    "additionalProperties": False
  }
}


# Example of input:
# "“Set ticket prices: London 799, Paris 899, Tokyo 1400, Berlin 499.”"
# “Update the ticket prices for these cities:
# Tokyo: 1400
# Berlin: 499”
# “Set prices: London=799, Paris=899.”


In [8]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function},
         {"type": "function", "function": set_price_function}]

In [9]:
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Get the price of a return ticket to the destination city.',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'set_ticket_price',
   'description': 'Set the prices of return tickets for multiple destination cities.',
   'parameters': {'type': 'object',
    'properties': {'ticket_prices': {'type': 'object',
      'description': 'A dictionary where the key is the destination city and the value is the ticket price.',
      'additionalProperties': {'type': 'string',
       'description': "The ticket price (formatted as a string, e.g., '$799')."}}},
    'required': ['ticket_prices'],
    'additionalProperties': False}}}]

In [10]:
DB = "prices.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)')
    conn.commit()

In [11]:
def get_ticket_price(city):
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        return f"Ticket price to {city} is ${result[0]}" if result else "No price data available for this city"

In [12]:
get_ticket_price("London")

DATABASE TOOL CALLED: Getting price for London


'Ticket price to London is $799.0'

In [13]:
def set_ticket_price(city, price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?', (city.lower(), price, price))
        conn.commit()

In [14]:
ticket_prices = {"london":799, "paris": 899, "tokyo": 1420, "sydney": 2999}
for city, price in ticket_prices.items():
    set_ticket_price(city, price)

In [15]:
get_ticket_price("Tokyo")

DATABASE TOOL CALLED: Getting price for Tokyo


'Ticket price to Tokyo is $1420.0'

In [23]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        if tool_name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            price_details = get_ticket_price(city)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
        # === Tool: set_ticket_price (multiple cities) ===
        elif tool_name == "set_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            ticket_prices = arguments.get("ticket_prices", {})

            # Process the dictionary however you store ticket prices
            # in memory storage
            # result = set_ticket_prices(ticket_prices)

            #  database sorage
            for city, price in ticket_prices.items():
                set_ticket_price(city, price) 
            
            responses.append({
                "role": "tool",
                "content": json.dumps({
                    "status": "success",
                    "updated": ticket_prices
                }),
                "tool_call_id": tool_call.id
            })

        else:
            responses.append({
                "role": "tool",
                "content": json.dumps({"error": "Unknown tool"}),
                "tool_call_id": tool_call.id
            })
    return responses

In [17]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    # solving issue where you ask for use tool 2 times like: "Send me the price to Londond, and only if the price is under $1000 please check Paris."
    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    return response.choices[0].message.content

In [24]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


DATABASE TOOL CALLED: Getting price for London
